# QwenImg 完整教程

这个 Notebook 覆盖了 QwenImg 的所有使用场景，方便你快速上手和调试。

## 📚 目录

1. [环境设置](#1-环境设置)
2. [文生图 - 基础用法](#2-文生图---基础用法)
3. [文生图 - 高级用法](#3-文生图---高级用法)
4. [图生视频 - 基础用法](#4-图生视频---基础用法)
5. [图生视频 - 高级用法](#5-图生视频---高级用法)
6. [文生视频](#6-文生视频)
7. [完整工作流](#7-完整工作流)
8. [模型列表](#8-模型列表)
9. [错误处理](#9-错误处理)
10. [实用技巧](#10-实用技巧)

## 1. 环境设置

### 安装依赖（如果还没安装）

In [ ]:
# 取消注释以安装依赖
# !pip install -q dashscope pillow requests python-dotenv

### 导入库

In [ ]:
import os
import sys
from pathlib import Path
from IPython.display import Image as IPImage, display, HTML
import json

# 添加项目路径（如果需要）
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from qwenimg import QwenImg

print("✅ QwenImg 导入成功！")

### 配置 API Key

**重要：** 获取 API Key: https://help.aliyun.com/zh/model-studio/get-api-key

有三种方式配置 API Key：
1. 环境变量：`export DASHSCOPE_API_KEY="sk-xxx"`
2. .env 文件：创建 `.env` 文件并添加 `DASHSCOPE_API_KEY=sk-xxx`
3. 直接在代码中传入（见下方）

In [ ]:
# 方式 1: 从环境变量读取（推荐）
client = QwenImg()

# 方式 2: 直接传入 API Key（不推荐用于生产环境）
# client = QwenImg(api_key="sk-your-api-key-here")

# 方式 3: 使用新加坡地域
# client = QwenImg(region="singapore")

print("✅ 客户端初始化成功！")

---

## 2. 文生图 - 基础用法

最简单的用法：3 行代码生成图片！

In [ ]:
# 生成一张图片
image = client.text_to_image("一只可爱的橘猫坐在窗台上")

# 显示图片
display(image)

print(f"✅ 图片生成成功！")
print(f"图片尺寸: {image.size}")
print(f"图片格式: {image.format}")
print(f"图片已保存到: ./outputs 目录")

---

## 3. 文生图 - 高级用法

### 3.1 自定义参数生成高质量图片

In [ ]:
# 使用详细的 prompt 和参数
image = client.text_to_image(
    prompt="""一位身穿白色长袍的古风男子，眉心有朱砂痣，长发飘逸，
    站在云雾缭绕的山间平台上，背景是金色的圆形光晕，
    古风仙侠风格，高品质4K，细节丰富""",
    negative_prompt="模糊、粗糙、色彩暗淡、风格杂乱",
    model="wan2.5-t2i-preview",
    size="1024*1024",
    seed=12345,  # 固定种子可重现结果
    prompt_extend=True,  # 自动扩展提示词
    watermark=False,
    output_dir="./outputs/advanced"
)

display(image)
print(f"✅ 高质量图片生成成功！尺寸: {image.size}")

### 3.2 生成多张图片

In [ ]:
# 一次生成 4 张图片
images = client.text_to_image(
    prompt="一只可爱的柴犬在樱花树下玩耍，春天，阳光明媚",
    n=4,  # 生成 4 张
    size="1024*1024",
    output_dir="./outputs/batch"
)

print(f"✅ 生成了 {len(images)} 张图片\n")

# 显示所有图片
for i, img in enumerate(images, 1):
    print(f"图片 {i}: {img.size}")
    display(img)

### 3.3 不同尺寸和方向

In [ ]:
# 正方形
square = client.text_to_image(
    prompt="一朵盛开的红玫瑰，微距摄影",
    size="1024*1024",  # 正方形
    output_dir="./outputs/sizes"
)

# 横版
landscape = client.text_to_image(
    prompt="美丽的日落海景，广角镜头",
    size="1280*720",  # 横版
    output_dir="./outputs/sizes"
)

# 竖版
portrait = client.text_to_image(
    prompt="一位优雅的女性肖像，古典油画风格",
    size="720*1280",  # 竖版
    output_dir="./outputs/sizes"
)

print("✅ 不同尺寸的图片生成成功！")
print(f"正方形: {square.size}")
print(f"横版: {landscape.size}")
print(f"竖版: {portrait.size}")

display(square)
display(landscape)
display(portrait)

### 3.4 不保存到磁盘，仅在内存处理

In [ ]:
# 不保存，只返回 PIL.Image 对象
image = client.text_to_image(
    prompt="一只蝴蝶停在花朵上",
    save=False,  # 不保存到磁盘
    return_pil=True
)

# 现在可以对图片进行各种处理
# 例如：调整大小、应用滤镜、裁剪等
resized = image.resize((512, 512))

display(image)
display(resized)

print("✅ 图片在内存中处理完成！")

---

## 4. 图生视频 - 基础用法

从图片生成视频

In [ ]:
# 首先生成一张图片
image = client.text_to_image(
    prompt="一只可爱的橘猫坐在窗台上，背景是美丽的花园",
    output_dir="./outputs/for_video"
)

display(image)

# 找到生成的图片文件路径
image_dir = Path("./outputs/for_video")
image_files = list(image_dir.glob("*.png"))
if image_files:
    image_path = str(image_files[-1])  # 使用最新的图片
    print(f"\n使用图片: {image_path}")
else:
    print("未找到图片文件！")
    image_path = None

In [ ]:
# 从图片生成视频
if image_path:
    video_url = client.image_to_video(
        image=image_path,
        prompt="橘猫缓缓转头看向窗外，微风吹动它的毛发",
        duration=10,  # 10 秒
        resolution="1080P"
    )
    
    print("\n✅ 视频生成成功！")
    print(f"📹 视频 URL: {video_url}")
    
    # 显示视频链接
    display(HTML(f'<a href="{video_url}" target="_blank">点击查看视频</a>'))

---

## 5. 图生视频 - 高级用法

### 5.1 详细的时间轴控制

In [ ]:
if image_path:
    video_url = client.image_to_video(
        image=image_path,
        prompt="""([锚定设定]，严格依据图片生成10秒视频，保持橘猫的特征和花园背景。)
        ([动态分层]，橘猫的耳朵轻轻抖动，尾巴缓缓摆动；
        花园中的花朵随微风摇曳；远处的树叶沙沙作响。)
        ([时间轴分层]，0-3秒：橘猫静静坐着；4-7秒：缓缓转头；8-10秒：看向远方。)
        ([技术参数]，帧率60帧每秒，保证流畅度。)""",
        negative_prompt="模糊、抖动、失真、不自然的动作",
        resolution="1080P",
        duration=10,
        seed=12345
    )
    
    print("\n✅ 详细控制的视频生成成功！")
    print(f"📹 视频 URL: {video_url}")
    display(HTML(f'<a href="{video_url}" target="_blank">点击查看视频</a>'))

### 5.2 不同分辨率对比

In [ ]:
if image_path:
    resolutions = ["480P", "720P", "1080P"]
    
    print("生成不同分辨率的视频...\n")
    
    for resolution in resolutions:
        video_url = client.image_to_video(
            image=image_path,
            prompt="橘猫眨眼睛",
            duration=5,
            resolution=resolution
        )
        
        print(f"✅ {resolution}: {video_url}")
        display(HTML(f'<a href="{video_url}" target="_blank">{resolution} 视频链接</a>'))

### 5.3 使用 Base64 编码（适合私密图片）

In [ ]:
if image_path:
    video_url = client.image_to_video(
        image=image_path,
        prompt="橘猫慢慢站起来伸懒腰",
        use_base64=True,  # 使用 Base64 编码
        duration=5,
        resolution="720P"
    )
    
    print("\n✅ 使用 Base64 编码的视频生成成功！")
    print(f"📹 视频 URL: {video_url}")
    display(HTML(f'<a href="{video_url}" target="_blank">点击查看视频</a>'))

### 5.4 使用在线图片 URL

In [ ]:
# 如果你有公开的图片 URL，可以直接使用
# 取消注释以使用：

# online_image_url = "https://example.com/your-image.png"
# video_url = client.image_to_video(
#     image=online_image_url,
#     prompt="描述视频内容",
#     duration=10
# )
# print(f"✅ 视频生成成功: {video_url}")

print("💡 提示：取消注释上面的代码，使用在线图片 URL 生成视频")

---

## 6. 文生视频

直接从文字生成视频，无需先生成图片

In [ ]:
# 基础用法
video_url = client.text_to_video(
    prompt="一只可爱的柴犬在草地上奔跑，阳光明媚，春天，高品质",
    duration=10,
    resolution="1080P"
)

print("✅ 视频生成成功！")
print(f"📹 视频 URL: {video_url}")
display(HTML(f'<a href="{video_url}" target="_blank">点击查看视频</a>'))

In [ ]:
# 高级用法：电影级效果
video_url = client.text_to_video(
    prompt="""一个充满动感的都市场景。
    夜晚，霓虹灯闪烁的街道上，一个穿着连帽衫的年轻人在奔跑。
    镜头跟随他的身影，展现城市的繁华与孤独。
    电影级光照，高能量，充满细节。""",
    negative_prompt="模糊、静止、低质量",
    resolution="1080P",
    duration=10,
    seed=12345
)

print("\n✅ 电影级视频生成成功！")
print(f"📹 视频 URL: {video_url}")
display(HTML(f'<a href="{video_url}" target="_blank">点击查看视频</a>'))

In [ ]:
# 自然风景
video_url = client.text_to_video(
    prompt="""美丽的日落场景，金色的阳光洒在平静的湖面上。
    微风吹过，湖面泛起涟漪。
    远处的山脉在晚霞中若隐若现。
    宁静、祥和的氛围，4K超清画质。""",
    resolution="1080P",
    duration=10
)

print("\n✅ 自然风景视频生成成功！")
print(f"📹 视频 URL: {video_url}")
display(HTML(f'<a href="{video_url}" target="_blank">点击查看视频</a>'))

---

## 7. 完整工作流

文生图 -> 图生视频的完整流程

In [ ]:
print("🎨 开始完整工作流：文生图 -> 图生视频")
print("=" * 60)

# 步骤 1：生成图片
print("\n📝 步骤 1: 生成图片...")
prompt_image = """一位身穿白色长袍的古风男子，
眉心有朱砂痣，长发飘逸，
站在云雾缭绕的山间平台上，
背景是金色的圆形光晕，
古风仙侠风格，高品质4K"""

image = client.text_to_image(
    prompt=prompt_image,
    negative_prompt="模糊、粗糙、色彩暗淡",
    size="1024*1024",
    seed=12345,
    output_dir="./outputs/workflow"
)

print("✅ 图片生成成功！")
display(image)

# 找到生成的图片
workflow_dir = Path("./outputs/workflow")
workflow_images = list(workflow_dir.glob("*.png"))
workflow_image_path = str(workflow_images[-1]) if workflow_images else None

if workflow_image_path:
    print(f"📁 图片保存在: {workflow_image_path}")
    
    # 步骤 2：生成视频
    print("\n🎬 步骤 2: 从图片生成视频...")
    prompt_video = """严格依据图片生成10秒视频，保持角色特征和场景风格。
    ([动态分层]，前景的云雾缓缓流动，角色的长发和衣袍随风轻轻摆动。
    远景的山峰和金色光晕保持稳定，营造神圣氛围。)
    ([时间轴分层]，0-3秒：展现整体场景；4-7秒：镜头微微推进，聚焦角色；
    8-10秒：镜头缓缓上移，展现天空和光晕。)
    ([技术参数]，60帧每秒，4K超清画质，保证流畅度。)"""
    
    video_url = client.image_to_video(
        image=workflow_image_path,
        prompt=prompt_video,
        negative_prompt="模糊、抖动、失真",
        resolution="1080P",
        duration=10,
        seed=12345
    )
    
    print("\n" + "=" * 60)
    print("🎉 工作流完成！")
    print("=" * 60)
    print(f"\n📸 图片: {workflow_image_path}")
    print(f"   尺寸: {image.size}")
    print(f"\n📹 视频: {video_url}")
    print(f"   分辨率: 1080P")
    print(f"   时长: 10 秒")
    
    display(HTML(f'<a href="{video_url}" target="_blank" style="font-size: 16px; color: #0066cc;">🎬 点击查看生成的视频</a>'))
else:
    print("❌ 未找到图片文件")

---

## 8. 模型列表

查看所有支持的模型

In [ ]:
# 查看所有模型
all_models = QwenImg.list_models()

print("所有支持的模型：")
print(json.dumps(all_models, indent=2, ensure_ascii=False))

In [ ]:
# 查看文生图模型
print("=" * 60)
print("文生图模型 (Text-to-Image)")
print("=" * 60)

t2i_models = QwenImg.list_models("t2i")
for model_id, info in t2i_models.items():
    print(f"\n📸 {model_id}")
    print(f"   名称: {info['name']}")
    print(f"   支持尺寸: {', '.join(info['supported_sizes'])}")
    print(f"   最大批次: {info['max_batch']}")

In [ ]:
# 查看图生视频模型
print("=" * 60)
print("图生视频模型 (Image-to-Video)")
print("=" * 60)

i2v_models = QwenImg.list_models("i2v")
for model_id, info in i2v_models.items():
    print(f"\n🎬 {model_id}")
    print(f"   名称: {info['name']}")
    print(f"   支持分辨率: {', '.join(info['supported_resolutions'])}")
    print(f"   支持时长: {info['supported_durations']} 秒")

In [ ]:
# 查看文生视频模型
print("=" * 60)
print("文生视频模型 (Text-to-Video)")
print("=" * 60)

t2v_models = QwenImg.list_models("t2v")
for model_id, info in t2v_models.items():
    print(f"\n🎥 {model_id}")
    print(f"   名称: {info['name']}")
    print(f"   支持分辨率: {', '.join(info['supported_resolutions'])}")
    print(f"   支持时长: {info['supported_durations']} 秒")

---

## 9. 错误处理

演示如何处理常见错误

In [ ]:
# 错误处理示例

# 1. API Key 错误
try:
    wrong_client = QwenImg(api_key="sk-wrong-key")
    image = wrong_client.text_to_image("测试")
except Exception as e:
    print(f"❌ API Key 错误: {e}")

# 2. 无效的模型
try:
    image = client.text_to_image(
        prompt="测试",
        model="invalid-model"
    )
except ValueError as e:
    print(f"\n❌ 模型错误: {e}")

# 3. 无效的尺寸
try:
    image = client.text_to_image(
        prompt="测试",
        size="invalid"
    )
except ValueError as e:
    print(f"\n❌ 尺寸错误: {e}")

# 4. 文件不存在
try:
    video_url = client.image_to_video(
        image="non_existent_file.png",
        prompt="测试"
    )
except FileNotFoundError as e:
    print(f"\n❌ 文件错误: {e}")

# 5. 批次数量超限
try:
    images = client.text_to_image(
        prompt="测试",
        n=10  # 超过最大值 4
    )
except ValueError as e:
    print(f"\n❌ 批次错误: {e}")

print("\n✅ 错误处理演示完成！")

---

## 10. 实用技巧

### 10.1 使用固定 seed 重现结果

In [ ]:
# 使用相同的 seed 可以生成相似的结果
prompt = "一只可爱的猫咪"
seed = 12345

print("生成两张使用相同 seed 的图片...\n")

image1 = client.text_to_image(prompt, seed=seed, save=False)
image2 = client.text_to_image(prompt, seed=seed, save=False)

print("图片 1:")
display(image1)

print("\n图片 2:")
display(image2)

print("\n💡 提示：使用相同的 seed 可以生成相似的结果")

### 10.2 批量生成不同风格

In [ ]:
# 批量生成不同风格的图片
base_prompt = "一只猫"
styles = [
    "水彩画风格",
    "油画风格",
    "赛博朋克风格",
    "古典素描风格"
]

print("生成不同风格的图片...\n")

for style in styles:
    full_prompt = f"{base_prompt}，{style}"
    print(f"🎨 {style}")
    
    image = client.text_to_image(
        prompt=full_prompt,
        save=False
    )
    
    display(image)
    print()

### 10.3 图片后处理

In [ ]:
# 生成图片并进行后处理
from PIL import ImageFilter, ImageEnhance

image = client.text_to_image(
    prompt="一朵玫瑰",
    save=False
)

print("原图:")
display(image)

# 调整亮度
enhancer = ImageEnhance.Brightness(image)
brighter = enhancer.enhance(1.5)
print("\n增强亮度:")
display(brighter)

# 应用模糊滤镜
blurred = image.filter(ImageFilter.GaussianBlur(radius=5))
print("\n高斯模糊:")
display(blurred)

# 调整大小
resized = image.resize((256, 256))
print("\n调整大小 (256x256):")
display(resized)

print("\n💡 提示：QwenImg 返回标准的 PIL.Image 对象，可以使用 Pillow 的所有功能！")

### 10.4 保存自定义格式

In [ ]:
# 生成图片并保存为不同格式
image = client.text_to_image(
    prompt="一只蝴蝶",
    save=False
)

# 创建输出目录
output_dir = Path("./outputs/custom_formats")
output_dir.mkdir(parents=True, exist_ok=True)

# 保存为不同格式
formats = {
    "PNG": "butterfly.png",
    "JPEG": "butterfly.jpg",
    "WEBP": "butterfly.webp"
}

for format_name, filename in formats.items():
    filepath = output_dir / filename
    if format_name == "JPEG":
        # JPEG 不支持透明度，需要转换
        rgb_image = image.convert('RGB')
        rgb_image.save(filepath, format=format_name, quality=95)
    else:
        image.save(filepath, format=format_name)
    
    print(f"✅ 保存为 {format_name}: {filepath}")

display(image)

---

## 🎉 教程完成！

你已经学会了 QwenImg 的所有功能：

✅ 文生图（基础和高级用法）  
✅ 图生视频（基础和高级用法）  
✅ 文生视频  
✅ 完整工作流  
✅ 模型列表查看  
✅ 错误处理  
✅ 实用技巧  

### 下一步

- 查看 `examples/` 目录中的更多示例脚本
- 阅读 README.md 了解完整文档
- 访问 [阿里云百炼文档](https://help.aliyun.com/zh/model-studio/) 了解更多

### 需要帮助？

- [API Key 获取](https://help.aliyun.com/zh/model-studio/get-api-key)
- [文生图文档](https://help.aliyun.com/zh/model-studio/text-to-image-v2-api-reference)
- [图生视频文档](https://help.aliyun.com/zh/model-studio/image-to-video-api-reference)

---

**Powered by Alibaba Cloud 百炼 & DashScope** 🚀